In [ ]:
import os
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor

from sklearn.linear_model import LinearRegression 
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression




In [ ]:

# Load the training and test data
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


In [ ]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'

row_ids = df_test['row ID']
df_test.drop(['row ID'], axis=1, inplace=True)

numeric_col = df_test.select_dtypes(include=["number"]).columns.tolist()
categorical_cols = df_test.select_dtypes(exclude=["number"]).columns.tolist()

# Label encode the "sub_area" column
label_encoder = LabelEncoder()
df_train['sub_area'] = label_encoder.fit_transform(df_train['sub_area'])
df_test['sub_area'] = label_encoder.transform(df_test['sub_area'])

# Create dummy variables for categorical features
X_train = pd.get_dummies(df_train.drop(columns=[target_col]), drop_first=True)
y = df_train[target_col]

X_test = pd.get_dummies(df_test, drop_first=True)


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
scaler = StandardScaler()
X_train[numeric_col] = scaler.fit_transform(X_train[numeric_col])
X_test[numeric_col] = scaler.transform(X_test[numeric_col])

print('X_train_scaled shape:', X_train.shape)
print('X_test_scaled shape:', X_test.shape)

all_columns = X_train.columns.tolist()

# Get the non-numeric columns by subtracting numeric_col from all_columns
non_numeric_columns = [col for col in all_columns if col not in numeric_col]


X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train.dtypes


In [ ]:

# Step 2: Use a Decision Tree Regressor to get the 100 most important features
tree_regressor = DecisionTreeRegressor(max_depth=8, random_state=42)
tree_regressor.fit(X_train, y)
importances = tree_regressor.feature_importances_


# Get indices of the top 100 features
top_100_feature_indices = np.argsort(importances)[-150:]

# Select the top 100 features
X_train = X_train.iloc[:, top_100_feature_indices]
X_test = X_test.iloc[:, top_100_feature_indices]


In [ ]:
selector = SelectKBest(score_func=f_regression, k=100)

X_train = selector.fit_transform(X_train,y)


X_test = selector.transform(X_test)

print(X_train.shape)
print(X_test.shape)

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test) 
print('X_train_scaled shape:', X_train.shape)
print('X_test_scaled shape:', X_test.shape)

In [ ]:
pca = PCA(n_components=100)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

print('X_train_pca shape:', X_train.shape)
print('X_test_pca shape:', X_test.shape)


In [ ]:
threshold = 1

selector = VarianceThreshold(threshold=threshold)

X_train = selector.fit_transform(X_train)
X_test = selector.transform(X_test)


print(X_train.shape)
print(X_test.shape)

In [ ]:
# Step 3: Use Forward Selection to get the first 10 best features
selector = SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=10)
selector.fit(X_train, y)

# Get the selected feature indices
selected_feature_indices = selector.get_support()


In [ ]:

# # Select the first 10 best features
# X_train = X_train.iloc[:, selected_feature_indices]
# X_test = X_test.iloc[:, selected_feature_indices]
# Select the first 10 best features
X_train = X_train[:, selected_feature_indices]
X_test = X_test[:, selected_feature_indices]

print('X_train_top10 shape:', X_train.shape)
print('X_test_top10 shape:', X_test.shape)



In [ ]:

# Step 4: Use Polynomial Features with interaction on the selected features
poly = PolynomialFeatures(degree=3, interaction_only=True)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

print('X_train_poly shape:', X_train.shape)
print('X_test_poly shape:', X_test.shape)


In [ ]:
print(X_train.shape)

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import RMSprop

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# Define your model with L2 regularization
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1)
])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Compile the model with RMSprop optimizer
optimizer = RMSprop(learning_rate=0.001)  # You can adjust the learning rate
model.compile(optimizer=optimizer,
              loss='mean_squared_error',
              metrics=[keras.metrics.RootMeanSquaredError(), 'mae'])

# Fit the model
model.fit(X_train, y, epochs=15, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])


In [ ]:

model = Sequential([
    keras.layers.Input(shape=(X_train.shape[1],)),
    
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(32, activation='relu'),
    
    keras.layers.Dense(1)
])

# Define EarlyStopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

model.compile(optimizer=custom_optimizer,
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError(), 'mae'])

model.fit(X_train, y, epochs=15, batch_size=32, verbose=1, validation_split=0.2, callbacks=[early_stopping])

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras.callbacks import EarlyStopping, ModelCheckpoint, LearningRateScheduler
from sklearn.metrics import mean_squared_error
from keras.optimizers import Adam, legacy
import math


# define the keras model
model = Sequential()
model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
#model.add(Dropout(0.2))  # Apply dropout with a rate of 0.2
model.add(Dense(64, activation='relu'))#sigmoid
# model.add(Dense(64, activation='relu'))#sigmoid
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Add regularizers to the dense layers
model.layers[1].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
model.layers[2].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
#model.layers[3].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01

opt = legacy.Adam(learning_rate=0.00021)  

# compile the keras model
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['RootMeanSquaredError'])
#apply early stoppping
early_stopping_monitor = EarlyStopping(patience=5)
#apply model checkpoint

#fit the keras model on the dataset
model.fit(X_train, y, epochs=50, batch_size=63, validation_split=0.2, callbacks=[early_stopping_monitor])

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

In [ ]:
test_predictions = model.predict(X_test).flatten()

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

result_df.to_csv('predictions_55.csv', index=False)




# Shahood




In [ ]:
import os
import torch
os.environ["KERAS_BACKEND"] = "torch"
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler, RobustScaler
from sklearn.linear_model import Lasso, Ridge
from sklearn.neural_network import MLPRegressor

from sklearn.linear_model import LinearRegression 
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE, VarianceThreshold
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, Ridge
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression




In [ ]:

# Load the training and test data
df_train_orig = pd.read_csv('train.csv')
df_test_orig = pd.read_csv('test.csv')


In [66]:

# Make a copy of the original data
df_train = df_train_orig.copy()
df_test = df_test_orig.copy()
target_col = 'price_doc'
row_id_col = 'row ID'

row_ids = df_test['row ID']
df_test.drop(['row ID'], axis=1, inplace=True)


X_train = df_train.drop(columns=[target_col])
y = df_train[target_col]



X_test = df_test


print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (181507, 271)
X_test shape: (77789, 271)


In [ ]:
features_to_exclude = [
    'ID_metro', 
    'ID_railroad_station_walk', 
    'ID_bus_terminal', 
    'cemetery_km', 
    'power_transmission_line_km', 
    'big_church_count_500', 
    'church_count_500', 
    'mosque_count_500', 
    'theater_km', 
    'museum_km'
]
X_train = X_train.drop(columns=features_to_exclude)
X_test = X_test.drop(columns=features_to_exclude)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Define a custom scoring function that returns -log(p-value)
def p_value_score(X, y):
    _, p_values = f_regression(X, y)
    return -np.log(p_values)

# Define the number of features you want to select
num_features_to_select = 250  # Adjust this number as per your requirement

# Initialize SelectKBest with the custom scoring function
selector = SelectKBest(score_func=p_value_score, k=num_features_to_select)

# Fit and transform X_train to select the best features
X_train_selected = selector.fit_transform(X_train, y)

# Get the indices of the selected features
selected_feature_indices = selector.get_support(indices=True)

# Get the names of the selected features
selected_feature_names = X_train.columns[selected_feature_indices]

# Transform X_test to keep only the selected features
X_test_selected = X_test.iloc[:, selected_feature_indices]

print('Selected Features:', selected_feature_names)
print('X_train_selected shape:', X_train_selected.shape)
print('X_test_selected shape:', X_test_selected.shape)


In [ ]:
X_train = X_train_selected
X_test = X_test_selected

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train = X_train.drop(columns=['sub_area'])
X_test = X_test.drop(columns=['sub_area'])

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
X_train = pd.get_dummies(X_train, drop_first=True)
X_test = pd.get_dummies(X_test, drop_first=True)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [67]:
# Identify categorical columns
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns

#'''
# Apply label encoding to each categorical column
for col in categorical_cols:
    le = LabelEncoder()
    # Fill missing values with a placeholder string
    #X_train[col] = X_train[col].fillna('Missing')
    X_train[col] = le.fit_transform(X_train[col])
    #X_test[col] = X_test[col].fillna('Missing')
    X_test[col] = le.transform(X_test[col])
    

In [68]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


print(X_train.shape[1])

271


In [ ]:
print(type(X_train))
print(type(X_test))

print(X_train.shape)
print(X_test.shape)

In [ ]:
dt_regressor = DecisionTreeRegressor(random_state=42, max_depth=6)
dt_regressor.fit(X_train, y)

# Calculate Feature Importance
feature_importance = dt_regressor.feature_importances_

# Select Important Features
num_features_to_keep = 50
top_feature_indices = feature_importance.argsort()[-num_features_to_keep:][::-1]
# X_train = X_train.iloc[:, top_feature_indices]
# X_test = X_test.iloc[:, top_feature_indices]
X_train = X_train[:, top_feature_indices]
X_test = X_test[:, top_feature_indices]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:
from sklearn.feature_selection import RFECV

dt_regressor = DecisionTreeRegressor(random_state=42, max_depth=4)

rfecv = RFECV(estimator=dt_regressor)

# Fit the RFECV selector to the training data
rfecv.fit(X_train, y)

# Get the indices of the selected features
selected_feature_indices = rfecv.support_

# Select the important features from the original dataset
X_train = X_train[:, selected_feature_indices]
X_test = X_test[:, selected_feature_indices]

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)


In [ ]:
from sklearn.feature_selection import VarianceThreshold

# Define a threshold for variance (adjust as needed)
variance_threshold = 1
# Initialize the Variancehreshold selector
selector = VarianceThreshold(threshold=variance_threshold)

# Fit and transform the selector on your selected feature matrix
X_train = selector.fit_transform(X_train)
X_test = selector.transform(X_test)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

In [ ]:

# Define the linear regression model
lr = LinearRegression()

# Forward Feature Selection
sfs = SequentialFeatureSelector(lr, n_features_to_select=9, direction='forward', scoring='neg_mean_squared_error', n_jobs=3)
sfs.fit(X_train, y)

# Get the mask of selected features
selected_features = sfs.get_support()



In [ ]:
poly = PolynomialFeatures(degree=3, interaction_only=True)
X_train = poly.fit_transform(X_train)
X_test = poly.transform(X_test)

print('X_train_poly shape:', X_train.shape)
print('X_test_poly shape:', X_test.shape)

In [ ]:
X_test = X_test.values


In [ ]:
X_train = X_train_selected
X_test = X_test_selected

print(X_train.shape)
print(X_test.shape)

In [ ]:
X_train
type(X_test)

In [ ]:
X_train = X_train[:, selected_features]
X_test = X_test[:, selected_features]

# Check the shape of the resulting X_train and X_test
print(X_train.shape)
print(X_test.shape)


In [ ]:
print(X_train.shape)
print(X_test.shape)

In [ ]:

# Perform PCA to reduce dimensionality to 240 components
n_components = 10
pca = PCA(n_components=n_components)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# Now X_train_pca and X_test_pca contain the top 240 principal components
print(X_train.shape[1])

In [70]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y, test_size=0.2, random_state=43)

In [71]:
print(X_train.shape)
print(X_val.shape)

(145205, 271)
(36302, 271)


In [ ]:
model = DecisionTreeRegressor(max_depth=8, random_state=42, min_samples_leaf=10)
model.fit(X_train, y_train)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

In [ ]:
import xgboost as xgb

# Create an XGBoost regressor
model = xgb.XGBRegressor(n_estimators=100, max_depth=10, random_state=42, learning_rate=0.1, min_child_weight=10)
model.fit(X_train, y_train)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

In [ ]:
# Create a Randomforest regressor
model = RandomForestRegressor(n_estimators=30, max_depth=8, random_state=42, min_samples_leaf=10, n_jobs=3, verbose=2)
model.fit(X_train, y_train)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

In [ ]:
import catboost as cb

model = cb.CatBoostRegressor(n_estimators=200, max_depth=10, random_state=42, learning_rate=0.01)
model.fit(X_train, y_train, verbose=0)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

In [ ]:
# Create a lightgbm regressor
import lightgbm as lgb

model = lgb.LGBMRegressor(n_estimators=370, max_depth=11, random_state=42, learning_rate=0.01)
model.fit(X_train, y_train)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

In [72]:
# Create a gradient boosting regressor
from sklearn.ensemble import GradientBoostingRegressor

model = GradientBoostingRegressor(n_estimators=100, max_depth=10, random_state=42, verbose=2)
model.fit(X_train, y_train)

# Make predictions on the validation data
val_predictions = model.predict(X_val)
print('RMSE:', np.sqrt(mean_squared_error(y_val, val_predictions)))

      Iter       Train Loss   Remaining Time 
         1 418297427500784.9375           38.70m
         2 367260811800726.8125           38.65m
         3 325640855320514.3125           38.64m
         4 291886516246745.6875           38.21m
         5 264693412456196.9375           37.69m
         6 241964868902013.5625           37.02m
         7 223859293356583.1875           36.41m
         8 208856638575196.7500           36.01m
         9 196299934994371.5938           35.55m
        10 186466365480101.3438           35.13m
        11 178428746853513.5000           34.74m
        12 170522264752746.8750           34.28m
        13 164315316680769.7812           33.86m
        14 158696967273114.2500           33.48m
        15 154342694160825.9688           33.21m
        16 150919711476288.1875           32.88m
        17 145975868633703.7500           32.75m
        18 143148971034507.6250           32.42m
        19 140303266571603.4062           32.17m
        20 137924274585

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Nadam
from keras.optimizers import legacy

# define the keras model
model = Sequential()

model.add(Dense(128, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dropout(0.2))  # Apply dropout with a rate of 0.2
# model.add(Dense(128, activation='relu'))#sigmoid
# model.add(Dense(128, activation='relu'))#sigmoid

model.add(Dense(64, activation='relu'))#sigmoid
# model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='linear'))

# Add regularizers to the dense layers
model.layers[1].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
model.layers[2].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
# model.layers[3].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
# model.layers[4].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01
#model.layers[3].kernel_regularizer = regularizers.l1(0.001)  # Apply L2 regularization with a factor of 0.01

optimizers = {
    "SGD": SGD(learning_rate=0.01, momentum=0.9),
    "RMSprop": RMSprop(learning_rate=0.01, rho=0.9),
    "Adagrad": Adagrad(learning_rate=0.01),
    "Adadelta": Adadelta(learning_rate=1.0, rho=0.95),
    "Nadam": Nadam(learning_rate=0.002, beta_1=0.9, beta_2=0.999)
}

opt = legacy.Adam(learning_rate=0.001)  
# opt = optimizers["RMSprop"]

# compile the keras model
model.compile(optimizer=opt, loss='mean_squared_error', metrics=['RootMeanSquaredError'])
#apply early stoppping
early_stopping_monitor = EarlyStopping(patience=5)
#apply model checkpoint

#fit the keras model on the dataset
model.fit(X_train, y_train, epochs=100, batch_size=128, validation_data=(X_val, y_val), callbacks=[early_stopping_monitor])

In [ ]:
#calculate rmse
y_val_pred = model.predict(X_val)
val_rmse = np.sqrt(mean_squared_error(y_val, y_val_pred))
print('Validation RMSE:', val_rmse)

In [ ]:
test_predictions = model.predict(X_test).flatten()

# Create a DataFrame with 'row ID' and predictions
result_df = pd.DataFrame({'row ID': row_ids, 'price_doc': test_predictions})

result_df.to_csv('predictions_94.csv', index=False)
